# Library

In [1]:
import pandas as pd
import numpy as np
import os

In [2]:
grade = 2
ods_folder = 'D:\과업\[한국환경공단]자동차_환경_빅데이터_구축_2차\data\ODS'
file_name = f'[ODS][G{grade}]등록정보&제원정보(말소적용)(한글컬럼)(2022.12.29).csv'
ods_path = os.path.join(ods_folder, file_name)

# 로드

In [3]:
# about 5m
df = pd.read_csv(ods_path, encoding='cp949', low_memory=False)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11207727 entries, 0 to 11207726
Data columns (total 59 columns):
 #   Column        Dtype  
---  ------        -----  
 0   차량관리번호        object 
 1   차량등록번호        object 
 2   본거지법정동코드      int64  
 3   소유자구분         object 
 4   차종_등록정보       object 
 5   용도            object 
 6   미정1_x         object 
 7   현재회원번호        object 
 8   미정2_x         object 
 9   미정3_x         object 
 10  차대번호          object 
 11  최초등록일자        int64  
 12  차량연식          int64  
 13  차량제작일자        float64
 14  배출가스검사유효일자    float64
 15  제원관리번호        object 
 16  배출가스인증번호      object 
 17  배출가스등급        object 
 18  저공해조치YN(미사용)  object 
 19  저공해조치종류       float64
 20  저공해조치구조변경YN   object 
 21  미정4           float64
 22  차량등록번호숫자정보    int64  
 23  차량말소YN        object 
 24  미정5           object 
 25  최초회원번호        object 
 26  취득일자          float64
 27  취득금액          float64
 28  미정6           object 
 29  생성일시_등록정보     object 
 30  수정일시_등록정보     ob

# 전처리
- 연료(제원정보) : 전기차 제외

In [4]:
df.columns

Index(['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '미정1_x',
       '현재회원번호', '미정2_x', '미정3_x', '차대번호', '최초등록일자', '차량연식', '차량제작일자',
       '배출가스검사유효일자', '제원관리번호', '배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류',
       '저공해조치구조변경YN', '미정4', '차량등록번호숫자정보', '차량말소YN', '미정5', '최초회원번호', '취득일자',
       '취득금액', '미정6', '생성일시_등록정보', '수정일시_등록정보', '미정7', '차명', '차종_제원정보', '차종분류',
       '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량',
       '엔진출력', '배기량', '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '미정1_y', '삭제YN',
       '승인일자', '구동형식', '변속기종류', '미정2_y', '생성일시_제원정보', '수정일시_제원정보', '미정3_y'],
      dtype='object')

In [5]:
df['연료'].unique()

array(['LGP(액화석유가스)', '휘발유', nan, 'CNG(압축천연가스)', '휘발유 하이브리드',
       'LNG(액화천연가스)', 'LPG 하이브리드', 'CNG 하이브리드'], dtype=object)

In [6]:
df1 = df[['차량관리번호', '차량등록번호', '본거지법정동코드', '소유자구분', '차종_등록정보', '용도', '현재회원번호', '차대번호', '최초등록일자', '차량연식', '차량제작일자', '배출가스검사유효일자',
          '제원관리번호', '배출가스인증번호', '배출가스등급', '저공해조치YN(미사용)', '저공해조치종류', '저공해조치구조변경YN', '차량등록번호숫자정보', '차량말소YN', '최초회원번호', '취득일자', '취득금액', 
          '생성일시_등록정보', '수정일시_등록정보', '차명', '차종_제원정보', '차종분류', '차종유형(소분류)', '자동차형식', '제작사명', '연료', '엔진형식', '공차중량', '총중량', '적재중량', '엔진출력', '배기량', 
          '승차인원', '차량길이', '차량너비', '차량높이', '원산지명', '삭제YN', '승인일자', '구동형식', '변속기종류', '생성일시_제원정보', '수정일시_제원정보', ]]

In [7]:
df1['연료'].unique()

array(['LGP(액화석유가스)', '휘발유', nan, 'CNG(압축천연가스)', '휘발유 하이브리드',
       'LNG(액화천연가스)', 'LPG 하이브리드', 'CNG 하이브리드'], dtype=object)

In [8]:
df1['연료'].value_counts()

휘발유            9871233
LGP(액화석유가스)    1171745
휘발유 하이브리드       140373
CNG(압축천연가스)      24307
LNG(액화천연가스)         25
CNG 하이브리드           17
LPG 하이브리드            1
Name: 연료, dtype: int64

In [9]:
df1.shape

(11207727, 49)

In [10]:
df2 = df1[df1['연료'] != '전기'].reset_index(drop=True)
df2.shape

(11207727, 49)

In [11]:
len(df2['차대번호'].unique())

11207677

## 중복 차대번호 처리
- '최초등록일자', '배출가스검사유효일자', ascending=False

In [12]:
df2 = df2.sort_values(['최초등록일자', '배출가스검사유효일자'], ascending=False).drop_duplicates('차대번호').reset_index(drop=True)
df2.shape

(11207677, 49)

### [출력]

In [13]:
# about 4m
df2.to_csv(os.path.join(ods_folder, f'[ODS][G{grade}]등록정보&제원정보(말소적용)(중복차대제거)(한글컬럼)(2022.12.29).csv'), index=False)

# 기초 분석

## [로드] 영문, 한글 컬럼명 

In [14]:
# 등록정보
ex1 = pd.read_excel('analysis/ODSvsSTD/[정제]CEG_CAR_MIG.xlsx')
# 제원정보
ex2 = pd.read_excel('analysis/ODSvsSTD/[정제]CEG_CAR_SRC_MIG.xlsx')

In [15]:
ex1['내용'] = ['수정일시_등록정보', '소유자구분', '차량등록번호', '본거지법정동코드', '생성일시_등록정보', '용도', '배출가스검사유효일자', '배출가스인증번호', '현재회원번호', '취득일자', '취득금액', 
               '최초회원번호', '최초등록일자', '차종_등록정보', '차량등록번호숫자정보', '차량관리번호', '차량제작일자', '차량연식', '차량말소YN', '차대번호', '제원관리번호', '배출가스등급', '저공해조치종류',
               '저공해조치구조변경YN', '저공해조치YN(미사용)', '미정', '미정', '미정', '미정', '미정', '미정', '미정', ]
ex2['내용'] = ['수정일시_제원정보', '승인일자', '삭제YN', '생성일시_제원정보', '총중량', '차종유형(소분류)', '차종분류', '차종_제원정보', '차명', '차량높이', '차량너비', '차량길이', '구동형식', '제작사명', 
               '제원관리번호', '적재중량', '자동차형식', '공차중량', '원산지명', '연료', '엔진형식', '엔진출력', '승차인원', '배기량', '변속기종류', '미정', '미정', '미정' ]
ex = pd.concat([ex1, ex2], ignore_index=True)

In [16]:
col_dict = {a:b for b, a in zip(ex['컬럼'], ex['내용'])}

In [17]:
an = df2.isnull().sum().reset_index()
an.columns = ['한글컬럼', '결측치']
an['컬럼'] = [col_dict[x] for x in an['한글컬럼']]
an = an[['컬럼', '한글컬럼', '결측치']]

## [출력] 등록정보&제원정보 결측치 통계

In [18]:
an.to_excel(f'analysis2/[BD3][G{grade}][통계]등록정보&제원정보_결측치(전기차제외)(말소적용)(중복차대제거).xlsx', index=False)

# 코드 마지막